Разработать модель, предсказывающую к какому из трех сегментов относится каждый клиент.

In [137]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_selection import mutual_info_classif

In [138]:
pd.set_option('display.max_columns', 10) # Снимаем ограничение по клонкам
df = pd.read_csv('contest_train.csv', sep=',') # Загрузка файла
prediction_data = pd.read_csv('contest_test.csv', sep=',')
# df.dropna(axis=1, how='any', inplace=True) # Удалеем колонны с пропусками

# Заменяем пропуски на median
df = df.fillna(df.median()) 
prediction_data = prediction_data.fillna(prediction_data.median())

In [139]:
pd.set_option('display.max_columns', 300) # Снимаем ограничение по клонкам
df = pd.read_csv('contest_train.csv', sep=',') # Загрузка файла

In [140]:
df['FEATURE_261'] = df["FEATURE_150"].fillna('нет').apply(lambda x: 1 if x == 'нет' else 0)
df['FEATURE_260'] = df["FEATURE_123"].fillna('нет').apply(lambda x: 1 if x == 'нет' else 0)
colnames = df.columns.tolist()
colnames = colnames[:-3] + colnames[:-4:-1]
df = df[colnames]


In [141]:
def feature_selector(df, columns_list=False, max_one_elemtnt_k = 0.9, max_nan_elemtnt_k = 0.7, max_corr = 0.9):
    '''
    Удаляем из df фичи которые имеют один уникальный элемент,
    много одинаковых элементов выше попрогового значения,
    пропущенные элементы выше порогового значения,
    Коррелирующие между собой фичи выше порогового значения. 
    '''
    drop_list = []
    max_one_elemtnt = df.count()[0] * max_one_elemtnt_k # Считаем границу количество уникального элемента
    max_nan_elemtnt = df.count()[0] * max_nan_elemtnt_k # Считаем границу пропусков в столбце

    if not columns_list: # если не добавил колонки для проверки, берем все, 
        columns = list(df.columns)[1:-1]    # либо настраиваем тут. 

    for col in columns: # первые 3-и проверки
        if (len(df[col].unique()) == 1) or \
           (df[col].value_counts().tolist()[0] > max_one_elemtnt) or \
           (df[col].isnull().sum() > max_nan_elemtnt):
            drop_list.append(col)

    df_corr  = df.corr().unstack().reset_index() # корреляции
    hig_corr = df_corr[(np.abs(df_corr[0]) > max_corr) & (df_corr['level_0'] != df_corr['level_1'])]
    couple   = hig_corr[['level_0', 'level_1']].values.tolist()
    
    while couple:
        _, two = couple[0]
        drop_list.append(two)
        for el in couple:
            if two in el:
                couple.pop(couple.index(el))

    return df.drop(columns=set(drop_list), axis=1)



In [143]:
def normal_colum(n):
    if n < Q1:
        return Q1 - 1
    elif n > Q3:
        return Q3 + 1
    return n

def normal_df(df):
    columns = list(df.columns)[1:-1]
    for col in columns:
        global Q1, Q3
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)

        if len(df[col].unique()) <= 2:
            df[col] = df[col].fillna(df[col].median())
        else:
            df[col] = df[col].apply(normal_colum)
            df[col] = df[col].fillna(df[col].min() - 1)
    return df


def normal_colum2(n):
    if n < Q1:
        return 0
    elif Q1 <= n < Q2:
        return 1
    elif Q2 <= n < Q3:
        return 2 
    return 3
def normal_df2(df):
    columns = list(df.columns)[1:-1]
    for col in columns:
        global Q1, Q2, Q3
        Q1 = df[col].quantile(0.25)
        Q2 = df[col].median()
        Q3 = df[col].quantile(0.75)

        if len(df[col].unique()) <= 10:
            df[col] = df[col].fillna(df[col].median())
        else:
            df[col] = df[col].apply(normal_colum2)
            df[col] = df[col].fillna(-1)
    return df

In [144]:
df = feature_selector(df)
df = normal_df2(df)

In [145]:
train_df = pd.concat([
    df[df['TARGET'] == 0].head(1200),
    df[df['TARGET'] == 1].head(1200),
    df[df['TARGET'] == 2].head(1200)
])

In [ ]:
columns = list(train_df.columns)[1:-1]
for col in columns:
    # train_df.boxplot(column=col)
    plt.figure(figsize=(10,7))
    bins = int(train_df[col].max()/10)
    plt.hist(train_df[col])
    plt.xlabel(col)
    plt.ylabel('count')
    plt.show()



In [124]:
df['targetBin'] = df["TARGET"].apply(lambda x: 0 if x in [0, 1] else 2).values # Для предсказания 2
myTrain = df.iloc[:15000, :] # На чем будем учиться
myTest  = df.iloc[15000:, :] # Для предсказания, для сравнения подходов
features = df.columns.tolist()[1:-2]

In [147]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
X = train_df[features]
y = train_df['targetBin']
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
features1 = featureScores.nlargest(100,'Score')['Specs'].tolist()

In [148]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X,y)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
features2 = list(feat_importances.nlargest(100).to_dict().keys())

In [149]:
features = set(features1) & set(features2)
len(features)

80

In [65]:
# Фичи которые будем использовать для Подходов
df_corr = df.corr().unstack().reset_index()
hig_corr_features = df_corr[(np.abs(df_corr[0]) > 0.15) & (df_corr['level_0'] != df_corr['level_1'])]
features = hig_corr_features[hig_corr_features['level_0'] == 'TARGET']['level_1'].values[1:-1]

In [146]:
features = train_df.columns.tolist()[1:-1]
df['targetBin'] = df["TARGET"].apply(lambda x: 0 if x in [0, 1] else 2).values
train_df['targetBin'] = train_df["TARGET"].apply(lambda x: 0 if x in [0, 1] else 2).values
myTrain  = train_df
myTest   = df

In [150]:
# Модель Random Forest сразу на классификацию
X = myTrain[features].values
y = myTrain['TARGET'].values
model_all = RandomForestClassifier()
model_all.fit(X, y)

# Тест + запись 
Xtest    = myTest[features].values
predict  = model_all.predict(Xtest)
myTest['tRFall'] = predict


C:\Users\PC\AppData\Local\Temp\ipykernel_12836\3336627446.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X = myTrain[features].values
C:\Users\PC\AppData\Local\Temp\ipykernel_12836\3336627446.py:8: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  Xtest    = myTest[features].values


In [151]:
# Модель для предсказание сначала 2-к потом 1-ц

two  = myTrain[myTrain['targetBin'] == 2]
zero = myTrain[myTrain['targetBin'] == 0].head(1000)
myTrainBin = pd.concat([two, zero])

In [152]:
X = myTrainBin[features].values
y = myTrainBin['targetBin'].values

model_search_two = RandomForestClassifier()
model_search_two.fit(X, y)

# Результат предсказания 2
Xtest = myTest[features].values
predict = model_search_two.predict(Xtest)
myTest['tRF01'] = predict

C:\Users\PC\AppData\Local\Temp\ipykernel_12836\3392727520.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X = myTrainBin[features].values
C:\Users\PC\AppData\Local\Temp\ipykernel_12836\3392727520.py:8: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  Xtest = myTest[features].values


In [153]:
# Находим 1
one  = myTrain[myTrain['TARGET'] == 1]
zero = myTrain[myTrain['TARGET'] == 0].head(3500)
myTrainbin01 = pd.concat([one, zero])
myTrainbin01['TARGET'].value_counts()

1    1200
0    1200
Name: TARGET, dtype: int64

In [154]:
X = myTrainbin01[features].values
y = myTrainbin01['TARGET'].values

model_search_one = RandomForestClassifier(n_estimators=100)
model_search_one.fit(X, y)

Xtest = myTest[features].values
predict = model_search_one.predict(Xtest)
myTest['TresultBin'] = predict

C:\Users\PC\AppData\Local\Temp\ipykernel_12836\3393674505.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X = myTrainbin01[features].values
C:\Users\PC\AppData\Local\Temp\ipykernel_12836\3393674505.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  Xtest = myTest[features].values


In [155]:
# объединяем результат работы поиска 1 и 2
result = []
list_ = list(myTest[['tRF01', 'TresultBin']].values)
for i, j in list_:
    result.append(2 if i == 2 else j)
myTest['TresultBin'] = result

In [156]:
# Сравнение 2-х вариантов 
print("Результат работы раздельно:")
print(classification_report(myTest['TARGET'], myTest['TresultBin']))

print("Результат поиска сразу всего:")
print(classification_report(myTest['TARGET'], myTest['tRFall']))

Результат работы раздельно:
              precision    recall  f1-score   support

           0       0.86      0.48      0.61     13029
           1       0.34      0.47      0.39      4237
           2       0.21      1.00      0.35      1124

    accuracy                           0.51     18390
   macro avg       0.47      0.65      0.45     18390
weighted avg       0.70      0.51      0.55     18390

Результат поиска сразу всего:
              precision    recall  f1-score   support

           0       0.86      0.52      0.65     13029
           1       0.38      0.59      0.46      4237
           2       0.29      1.00      0.45      1124

    accuracy                           0.56     18390
   macro avg       0.51      0.70      0.52     18390
weighted avg       0.71      0.56      0.59     18390



In [33]:
# Результат предсказания 2
X1 = prediction_data[features].values
predict = model_search_two.predict(X1)
prediction_data['TARGETpred'] = predict

X2 = prediction_data[features].values
predict = model_search_one.predict(X2)
prediction_data['TARGET'] = predict

result = []
list_ = list(prediction_data[['TARGETpred', 'TARGET']].values)
for i, j in list_:
    result.append(2 if i == 2 else j)
prediction_data['TARGET'] = result


In [34]:
prediction_data

,ID,FEATURE_0,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,FEATURE_8,FEATURE_9,FEATURE_10,FEATURE_11,FEATURE_12,FEATURE_13,FEATURE_14,FEATURE_15,FEATURE_16,FEATURE_17,FEATURE_18,FEATURE_19,FEATURE_20,FEATURE_21,FEATURE_22,FEATURE_23,FEATURE_24,FEATURE_25,FEATURE_26,FEATURE_27,FEATURE_28,FEATURE_29,FEATURE_30,FEATURE_31,FEATURE_32,FEATURE_33,FEATURE_34,FEATURE_35,FEATURE_36,FEATURE_37,FEATURE_38,FEATURE_39,FEATURE_40,FEATURE_41,FEATURE_42,FEATURE_43,FEATURE_44,FEATURE_45,FEATURE_46,FEATURE_47,FEATURE_48,FEATURE_49,FEATURE_50,FEATURE_51,FEATURE_52,FEATURE_53,FEATURE_54,FEATURE_55,FEATURE_56,FEATURE_57,FEATURE_58,FEATURE_59,FEATURE_60,FEATURE_61,FEATURE_62,FEATURE_63,FEATURE_64,FEATURE_65,FEATURE_66,FEATURE_67,FEATURE_68,FEATURE_69,FEATURE_70,FEATURE_71,FEATURE_72,FEATURE_73,FEATURE_74,FEATURE_75,FEATURE_76,FEATURE_77,FEATURE_78,FEATURE_79,FEATURE_80,FEATURE_81,FEATURE_82,FEATURE_83,FEATURE_84,FEATURE_85,FEATURE_86,FEATURE_87,FEATURE_88,FEATURE_89,FEATURE_90,FEATURE_91,FEATURE_92,FEATURE_93,FEATURE_94,FEATURE_95,FEATURE_96,FEATURE_97,FEATURE_98,FEATURE_99,FEATURE_100,FEATURE_101,FEATURE_102,FEATURE_103,FEATURE_104,FEATURE_105,FEATURE_106,FEATURE_107,FEATURE_108,FEATURE_109,FEATURE_110,FEATURE_111,FEATURE_112,FEATURE_113,FEATURE_114,FEATURE_115,FEATURE_116,FEATURE_117,FEATURE_118,FEATURE_119,FEATURE_120,FEATURE_121,FEATURE_122,FEATURE_123,FEATURE_124,FEATURE_125,FEATURE_126,FEATURE_127,FEATURE_128,FEATURE_129,FEATURE_130,FEATURE_131,FEATURE_132,FEATURE_133,FEATURE_134,FEATURE_135,FEATURE_136,FEATURE_137,FEATURE_138,FEATURE_139,FEATURE_140,FEATURE_141,FEATURE_142,FEATURE_143,FEATURE_144,FEATURE_145,FEATURE_146,FEATURE_147,FEATURE_148,FEATURE_149,FEATURE_150,FEATURE_151,FEATURE_152,FEATURE_153,FEATURE_154,FEATURE_155,FEATURE_156,FEATURE_157,FEATURE_158,FEATURE_159,FEATURE_160,FEATURE_161,FEATURE_162,FEATURE_163,FEATURE_164,FEATURE_165,FEATURE_166,FEATURE_167,FEATURE_168,FEATURE_169,FEATURE_170,FEATURE_171,FEATURE_172,FEATURE_173,FEATURE_174,FEATURE_175,FEATURE_176,FEATURE_177,FEATURE_178,FEATURE_179,FEATURE_180,FEATURE_181,FEATURE_182,FEATURE_183,FEATURE_184,FEATURE_185,FEATURE_186,FEATURE_187,FEATURE_188,FEATURE_189,FEATURE_190,FEATURE_191,FEATURE_192,FEATURE_193,FEATURE_194,FEATURE_195,FEATURE_196,FEATURE_197,FEATURE_198,FEATURE_199,FEATURE_200,FEATURE_201,FEATURE_202,FEATURE_203,FEATURE_204,FEATURE_205,FEATURE_206,FEATURE_207,FEATURE_208,FEATURE_209,FEATURE_210,FEATURE_211,FEATURE_212,FEATURE_213,FEATURE_214,FEATURE_215,FEATURE_216,FEATURE_217,FEATURE_218,FEATURE_219,FEATURE_220,FEATURE_221,FEATURE_222,FEATURE_223,FEATURE_224,FEATURE_225,FEATURE_226,FEATURE_227,FEATURE_228,FEATURE_229,FEATURE_230,FEATURE_231,FEATURE_232,FEATURE_233,FEATURE_234,FEATURE_235,FEATURE_236,FEATURE_237,FEATURE_238,FEATURE_239,FEATURE_240,FEATURE_241,FEATURE_242,FEATURE_243,FEATURE_244,FEATURE_245,FEATURE_246,FEATURE_247,FEATURE_248,FEATURE_249,FEATURE_250,FEATURE_251,FEATURE_252,FEATURE_253,FEATURE_254,FEATURE_255,FEATURE_256,FEATURE_257,FEATURE_258,FEATURE_259,TARGET,TARGETpred,TARGET2
0,84728433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.265322,110.813724,0.0,0.0,1.0,154.726900,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,-13.857818,0.0,61.287620,30.607010,0.0,35.720469,0.0,0.0,0.0,0.0,0.0,0.0,-13.363083,27.042315,121.164445,410.777662,418.184730,288.779844,0.0,0.0,0.0,0.0,0.0,0.0,9.814280,127.316255,140.253660,224.746131,165.411893,475.424812,26.583890,146.018837,265.737867,604.491503,659.827819,800.933376,49.935832,129.457907,151.852115,101.333077,58.323627,279.256776,67.445173,235.620404,363.524021,469.295854,313.095897,523.738616,28.0,29.0,11.115539,-4.006499,-13.473595,1.0,0.0,0.096774,79.023306,143.448293,562.376815,37.242363,157.809322,83.678082,89.993600,351.394700,1.210967,-0.335548,5.621130,41.154574,2.256960,15.278167,22.153717,63.346740,238.329810,15.212931,0.273242,2.720437,20.491297,21.948040,4.242742,0.716410,7.035782,82.186024,78.854780,10.822625,0.128304,1.059200,12.566417,26.107138,10.843584,0.000000,1.333333,22.819023,31.817500,17.3

In [35]:
prediction_data[['ID', 'TARGET']].to_csv('Sergunin.csv', sep=',', index=False)

Для решения задачи использовал RandomForest поскольку она дала лучшие результаты на первичном прогоне среди других моделек классификации. 
Использовал 2-а варианта классификации.
1) стандартна сегментировал по 3-м категориям.
2) Нашел с начала 2-ю категорию поскольку она сильнее всего отличается от остальных,
    После чего уже между остальными искал 0 и 1. 
    Вариант оказался не лучше рандома. Его я оставил хорошенько обрезов, в качестве сравнения! 

LogisticRegression выдал хороший результат однако перенести его не получилось

